In [1]:
import re
import os
import time
from nltk.stem import PorterStemmer 

In [2]:
#==============================
def ps_stem(word_list):
    ps = PorterStemmer()
    p=[]
    for w in word_list:
        p.append(ps.stem(w))
    return(p)
#==============================
def process_files(path, filenames):
    file_to_terms = {}
    for file in filenames:
        pattern = re.compile('[\W_\d]+')
        file_to_terms[file] = open(path+file, 'r', encoding="ANSI").read().lower();
        file_to_terms[file] = pattern.sub(' ',file_to_terms[file])
        re.sub(r'[\W_\d]+','', file_to_terms[file])
        file_to_terms[file] = file_to_terms[file].split()
        file_to_terms[file] = ps_stem(file_to_terms[file]) #ps_stemmer
    return file_to_terms

def index_one_file(termlist):
    fileIndex = {}
    for index, word in enumerate(termlist):
        if word in fileIndex.keys():
            fileIndex[word].append(index)
        else:
            fileIndex[word] = [index]
    return fileIndex

def make_indices(termlists):
    total = {}
    for filename in termlists.keys():
        total[filename] = index_one_file(termlists[filename])
    return total

def fullIndex(regdex):
    total_index = {}
    for filename in regdex.keys():
        for word in regdex[filename].keys():
            if word in total_index.keys():
                if filename in total_index[word].keys():
                    total_index[word][filename].extend(regdex[filename][word][:])
                else:
                    total_index[word][filename] = regdex[filename][word]
            else:
                total_index[word] = {filename: regdex[filename][word]}
    return total_index
#===========================
def get_filenames(path):
    filenames=[]
    for f in os.listdir(path):
        filenames.append(f)
    filenames.sort(key = lambda i:int(re.search(r'(\d+)',i).group())) #照檔名數字排
    return(filenames)

def inv_dict_sort(inv_index):
    inv_index_sort={}
    for key in sorted(inv_index.keys()):
        inv_index_sort[key] = inv_index[key]
    return(inv_index_sort)

#for add document
def sort_dict_by_docname(d):
    keys = list(d)
    keys.sort(key = lambda i:int(re.search(r'(\d+)',i).group())) #照檔名數字排
    temp_dict={}
    for i in keys:
        temp_dict[i]=d[i]
    return(temp_dict)
#===========================

#Remove document
def rm_doc(path, doclist, inv_index):
    for docname in doclist:
        item = make_indices(process_files(path, [docname]))
        terms = item[docname].keys()
        for i in terms:
            inv_index[i].pop(docname, None)
    return(inv_index)

#Add document
def add_doc(path, doclist, inv_index):
    for docname in doclist:
        item = make_indices(process_files(path, [docname]))
        terms = item[docname].keys()
        for i in terms:
            try:
                inv_index[i].update( {docname : item[docname][i]} )
            except:
                inv_index[i]={docname : item[docname][i]}
            inv_index[i] = sort_dict_by_docname(inv_index[i])  
    return(inv_index)
#===========================

def one_word_query(word, invertedIndex):
    ps = PorterStemmer()
    pattern = re.compile('[\W_\d]+')
    word = pattern.sub(' ',word).lower()
    word = ps.stem(word) #stemmer
    if word in invertedIndex.keys():
        return [filename for filename in invertedIndex[word].keys()]
    else:
        return []

def free_text_query(string):
    pattern = re.compile('[\W_]+')
    string = pattern.sub(' ',string)
    result = []
    for word in string.split():
        result += one_word_query(word,inv_index)
    return list(set(result))

#===========================    
def AND(list1, list2, pt=False):
    answer=[]
    i=iter(list1)
    j=iter(list2)
    if list1 and list2:
        p1=next(i)
        p2=next(j)
        while True:
            try:
                #正規表達式
                n1 = int(re.search(r'(\d+)',p1).group())
                n2 = int(re.search(r'(\d+)',p2).group())
                if(pt):
                    print('p1 %s' % p1)
                    print('p2 %s' % p2)
                    print('---')
                if(n1==n2):
                    answer.append(p1)
                    p1=next(i)
                    p2=next(j)
                elif(n1<n2):
                    p1=next(i)
                else:
                    p2=next(j)
            except StopIteration:
                break
    return(answer)
#=========================== 

def AND_EASY(list1,list2):
    ListA = list()
    for x in list1:
        for y in list2:
            if(x == y):
                ListA.append(x)
    
    return ListA    

#=========================== 
def phrase_query(string, invertedIndex):
    ps = PorterStemmer()
    pattern = re.compile('[\W_\d]+')
    string = pattern.sub(' ',string)
    listOfLists, result = [],[]
    for word in string.split():
        word = ps.stem(word) #stemmer
        listOfLists.append(one_word_query(word, invertedIndex))
    setted = set(listOfLists[0]).intersection(*listOfLists)
    for filename in setted:
        temp = []
        for word in string.split():
            word = ps.stem(word) #stemmer
            try:
                temp.append(invertedIndex[word][filename][:])
            except:
                continue
        for i in range(len(temp)):
            for ind in range(len(temp[i])):
                temp[i][ind] -= i
        if set(temp[0]).intersection(*temp):
            result.append(filename)
    #return rankResults(result, string)
    result.sort(key = lambda i:int(re.search(r'(\d+)',i).group())) #照檔名數字排
    return result

In [3]:
#主執行區
path = './data/txt/'
filenames = get_filenames(path)
inv_index = fullIndex(make_indices(process_files(path, filenames)))
inv_index = inv_dict_sort(inv_index)

In [15]:
#移除文件
path = './data/txt/'
doclist = ['d (3).txt','d (12).txt']
inv_index = rm_doc(path, doclist, inv_index)

In [17]:
#加入文件
path = './data/txt/'
doclist = ['d (3).txt','d (12).txt']
inv_index = add_doc(path, doclist, inv_index)

In [18]:
inv_index['constant']

{'d (1).txt': [115, 122, 194],
 'd (2).txt': [87],
 'd (3).txt': [14, 26, 80, 127, 134, 147, 265, 300, 331, 362, 418, 458],
 'd (12).txt': [37],
 'd (47).txt': [352],
 'd (116).txt': [228, 246, 260],
 'd (158).txt': [24],
 'd (184).txt': [447],
 'd (185).txt': [296],
 'd (188).txt': [463],
 'd (204).txt': [348],
 'd (245).txt': [811],
 'd (250).txt': [212, 259],
 'd (251).txt': [40],
 'd (252).txt': [6, 264],
 'd (253).txt': [217, 357],
 'd (254).txt': [16],
 'd (296).txt': [228, 1067],
 'd (317).txt': [41],
 'd (385).txt': [57],
 'd (391).txt': [112],
 'd (418).txt': [103, 217, 471, 677, 826],
 'd (419).txt': [147, 167, 192],
 'd (504).txt': [507],
 'd (508).txt': [175],
 'd (545).txt': [208],
 'd (585).txt': [179],
 'd (608).txt': [331],
 'd (634).txt': [565],
 'd (640).txt': [259],
 'd (652).txt': [514, 553],
 'd (663).txt': [517],
 'd (667).txt': [551, 572],
 'd (677).txt': [575],
 'd (683).txt': [259],
 'd (718).txt': [9],
 'd (754).txt': [710, 761, 836],
 'd (755).txt': [500],
 '

In [ ]:
#=======================================================

In [25]:
#單字查詢
query = 'p2p'
print(one_word_query(query,inv_index))

[]


In [26]:
#連續字查詢
q1 = phrase_query('skyline query',inv_index)
print(q1)

['d (147).txt', 'd (148).txt', 'd (149).txt', 'd (150).txt', 'd (151).txt', 'd (159).txt', 'd (167).txt', 'd (168).txt', 'd (169).txt', 'd (170).txt', 'd (171).txt', 'd (172).txt', 'd (303).txt', 'd (304).txt', 'd (305).txt', 'd (306).txt', 'd (308).txt', 'd (309).txt', 'd (1010).txt', 'd (1011).txt', 'd (1012).txt', 'd (1015).txt', 'd (1062).txt', 'd (1063).txt', 'd (1064).txt', 'd (1070).txt', 'd (1074).txt']


In [9]:
#===AND_EASY===
start_time = time.time()

q1 = one_word_query('this',inv_index)
q2 = one_word_query('we',inv_index)
intersection = AND_EASY(q1,q2)
print('q1_len:%s' % len(q1))
print('q2_len:%s' % len(q2))
print('AND_len:%s' % len(intersection))

end_time = time.time()
print("AND_EASY : --- %s seconds ---\n\n" % (end_time - start_time))

#===AND===
start_time = time.time()

q1 = one_word_query('this',inv_index)
q2 = one_word_query('we',inv_index)
intersection = AND(q1,q2)
print('q1_len:%s' % len(q1))
print('q2_len:%s' % len(q2))
print('AND_len:%s' % len(intersection))

end_time = time.time()
print("AND : --- %s seconds ---\n\n" % (end_time - start_time))

#===AND_HASH===
start_time = time.time()

q1 = set(one_word_query('we',inv_index))
q2 = set(one_word_query('this',inv_index))
intersection = list(q1 & q2) # '&' operator is used for set intersection
print('q1_len:%s' % len(q1))
print('q2_len:%s' % len(q2))
print('AND_len:%s' % len(intersection))

end_time = time.time()
print("AND_HASH : --- %s seconds ---\n\n" % (end_time - start_time))

#===OR_HASH===
start_time = time.time()

q1 = set(one_word_query('we',inv_index))
q2 = set(one_word_query('this',inv_index))
union = list(q1 | q2) # '&' operator is used for set intersection
print('q1_len:%s' % len(q1))
print('q2_len:%s' % len(q2))
print('OR_len:%s' % len(union))

end_time = time.time()
print("OR_HASH : --- %s seconds ---\n\n" % (end_time - start_time))

#===NOT_HASH===
start_time = time.time()

q1 = set(one_word_query('we',inv_index))
q2 = set(one_word_query('this',inv_index))
Complement = list(q1 - q2) # '&' operator is used for set intersection
print('q1_len:%s' % len(q1))
print('q2_len:%s' % len(q2))
print('NOT_len:%s' % len(Complement))

end_time = time.time()
print("NOT_HASH : --- %s seconds ---" % (end_time - start_time))

q1_len:891
q2_len:878
AND_len:749
AND_EASY : --- 0.024001121520996094 seconds ---


q1_len:891
q2_len:878
AND_len:749
AND : --- 0.004000186920166016 seconds ---


q1_len:878
q2_len:891
AND_len:749
AND_HASH : --- 0.0010001659393310547 seconds ---


q1_len:878
q2_len:891
OR_len:1020
OR_HASH : --- 0.0 seconds ---


q1_len:878
q2_len:891
NOT_len:129
NOT_HASH : --- 0.0010001659393310547 seconds ---


In [10]:
query = free_text_query('base-station')
print(len(query))

496


In [11]:
def q(text):
    text = set(one_word_query(text,inv_index))
    return text

query = q('base') | q('station')
print(len(query))

query = q('base') - q('station')
print(len(query))

query = q('base') & q('station')
print(len(query))

496
454
38
